Build and train a graph convolutional neural network using PyTorch Geometric for the node property prediction task.

We will use ogbn-products dataset.

## OGBN-Products

The ogbn-products dataset is an undirected and unweighted graph, representing an Amazon product co-purchasing network. Nodes represent products sold in Amazon, and edges between two products indicate that the products are purchased together. Node features are generated by extracting bag-of-words features from the product descriptions followed by a Principal Component Analysis to reduce the dimension to 100.

The task is to predict the category of a product in a multi-class classification setup, where the 47 top-level categories are used for target labels.

In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.0.1+cu118


Download the necessary packages for PyG. Make sure that your version of torch matches the output from the cell above. In case of any issues, more information can be found on the [PyG's installation page](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html).

In [2]:
# Install torch geometric
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{torch.__version__}.html
!pip install torch-geometric
!pip install ogb

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 24.0 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910460 sha256=e5388d85cfb824bc3b6dc400be27c9977a8f7c874da5dd25398c2e13a0852807
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=

In [3]:
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
import numpy as np
from torch_geometric.typing import SparseTensor
import torch.nn as nn

## Load and Preprocess the Dataset

In [4]:
dataset_name = 'ogbn-products'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
data = dataset[0]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))


This will download 1.38GB. Will you proceed? (y/N)
y


Downloaded 1.38 GB: 100%|██████████| 1414/1414 [00:26<00:00, 53.41it/s]


Extracting dataset/products.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


...

Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 87.15it/s]


Saving...


Done!


Device: cuda


In [5]:
data

Data(num_nodes=2449029, x=[2449029, 100], y=[2449029, 1], adj_t=[2449029, 2449029, nnz=123718280])

This dataset is very big and if you try to run it as it is on colab, you may get an out of memory error.

One solution is to use batching and train on subgraphs. Here, we will just make a smaller dataset so that we can train it in one go.

In [6]:
# We need to have edge indxes to make a subgraph. We can get those from the adjacency matrix.
data.edge_index = torch.stack([data.adj_t.__dict__["storage"]._row, data.adj_t.__dict__["storage"]._col])

# We will only use the first 100000 nodes.
sub_nodes = 100000
sub_graph = data.subgraph(torch.arange(sub_nodes))

# Update the adjaceny matrix according to the new graph
sub_graph.adj_t = SparseTensor(
    row=sub_graph.edge_index[0],
    col=sub_graph.edge_index[1],
    sparse_sizes=None,
    is_sorted=True,
    trust_data=True,
)

sub_graph = sub_graph.to(device)

data = sub_graph
data = data.to(device)


In [7]:
# Spilt data into train validation and test set
split_sizes = [int(sub_nodes*0.8),int(sub_nodes*0.05),int(sub_nodes*0.15)]
indices = torch.arange(sub_nodes)
np.random.shuffle(indices.numpy())
split_idx = {s:t for t,s in zip(torch.split(indices, split_sizes, dim=0), ["train", "valid", "test"])}
split_idx

{'train': tensor([88473, 12137, 66526,  ..., 19282,  2687, 92877]),
 'valid': tensor([87559, 71191,  1913,  ..., 85485, 80590, 14361]),
 'test': tensor([49367, 69282, 89334,  ..., 55099, 66480, 47161])}

In [8]:
train_idx = split_idx['train'].to(device)

In [9]:
print(f"Feature Length of each node: {data.x.shape[1]}")

Feature Length of each node: 100


## GCN Model

Now we will implement our GCN model!

In [27]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, return_embeds=False):
        super(GCN, self).__init__()
        self.gcnlayers = nn.ModuleList([
            GCNConv(input_dim, hidden_dim),
            GCNConv(hidden_dim, hidden_dim),
            GCNConv(hidden_dim, hidden_dim),
            GCNConv(hidden_dim, hidden_dim)
        ])
        self.last_gcn = GCNConv(hidden_dim, output_dim)
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.BatchNorm1d(hidden_dim)
        ])
        self.softmax = nn.LogSoftmax()
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.gcnlayers:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):

        for i in range(len(self.gcnlayers)):
            x = self.gcnlayers[i](x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        x = self.last_gcn(x, adj_t)

        if not self.return_embeds:
          x = self.softmax(x)
        return x


In [28]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    loss = 0

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y[train_idx].reshape(-1))

    loss.backward()
    optimizer.step()

    return loss.item()

In [29]:

@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    model.eval()

    out = model(data.x, data.adj_t)

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      df.to_csv('ogbn-products_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [33]:
args = {
    'device': device,
    'hidden_dim': 256,
    'lr': 0.01,
    'epochs': 200,
}

In [34]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes).to(device)
evaluator = Evaluator(name='ogbn-products')

In [35]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])


best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  model.train()
  optimizer.zero_grad()
  out = model(data.x, data.adj_t)
  loss = F.nll_loss(out[train_idx], data.y[train_idx].reshape(-1))
  loss.backward()
  optimizer.step()
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss.item():.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

<ipython-input-27-7c3925edf625>:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


Epoch: 01, Loss: 4.2937, Train: 24.46%, Valid: 25.30% Test: 23.94%
Epoch: 02, Loss: 2.2312, Train: 67.28%, Valid: 66.30% Test: 67.13%
Epoch: 03, Loss: 1.2245, Train: 71.13%, Valid: 69.72% Test: 71.21%
Epoch: 04, Loss: 0.9818, Train: 79.23%, Valid: 78.52% Test: 79.18%
Epoch: 05, Loss: 0.8637, Train: 81.38%, Valid: 80.50% Test: 81.49%
Epoch: 06, Loss: 0.7992, Train: 82.27%, Valid: 81.16% Test: 82.23%
Epoch: 07, Loss: 0.7397, Train: 83.43%, Valid: 82.76% Test: 83.51%
Epoch: 08, Loss: 0.6968, Train: 84.69%, Valid: 84.26% Test: 84.63%
Epoch: 09, Loss: 0.6703, Train: 85.44%, Valid: 84.80% Test: 85.20%
Epoch: 10, Loss: 0.6473, Train: 85.79%, Valid: 84.90% Test: 85.53%
Epoch: 11, Loss: 0.6284, Train: 85.79%, Valid: 85.00% Test: 85.62%
Epoch: 12, Loss: 0.6139, Train: 85.91%, Valid: 85.14% Test: 85.88%
Epoch: 13, Loss: 0.5939, Train: 86.17%, Valid: 85.46% Test: 85.94%
Epoch: 14, Loss: 0.5842, Train: 86.27%, Valid: 85.54% Test: 86.04%
Epoch: 15, Loss: 0.5730, Train: 86.48%, Valid: 85.86% Test: 86